In [ ]:
# Imports
import cv2
import base64
import numpy as np
import io
from PIL import Image
import keras
from keras import backend as k
from keras.models import Sequential, load_model
from keras.preprocessing.image import ImageDataGenerator
from flask import request, jsonify, Flask
import numpy as np
import cv2
import matplotlib.pyplot as plt
from facenet_pytorch import MTCNN


# App
app = Flask(__name__)

# ------------------------------------------------------------

# function to load the models
def load_model_fn(path):
    model = load_model(path)
    print('Model loaded!')
    return model
# ------------------------------------------------------------

# function to resize our image
def preprocess_image(image, target_size):
    if image.mode != 'RGB':
        image = image.convert('RGB')
    image = image.resize(target_size)
    image = np.array(image)
    image = np.expand_dims(image, axis=0)
    return image
# ------------------------------------------------------------

# Initialize MTCNN
mtcnn = MTCNN()
# Crop Face Funtion:
def crop_face(image_array):
    IMG_SIZE = 145
    
    # Use MTCNN to detect faces
    boxes, _ = mtcnn.detect(image_array)
    
    # Check if a face was detected
    if boxes is not None:
        for box in boxes:
            x, y, w, h = box
            
            # Crop and resize the image
            roi_color = image_array[int(y):int(h), int(x):int(w)]
            try:
                resized_array = cv2.resize(roi_color, (IMG_SIZE, IMG_SIZE))
            except:
                return None
            
            return resized_array

    return None  # Return None if no faces are detected
# ------------------------------------------------------------

# Crop Eye Funtion:
def crop_eye(image_array):
    # Initialize MTCNN
    mtcnn = MTCNN(keep_all=True)
    IMG_SIZE = 145
    
    # Use MTCNN to detect faces and facial landmarks
    boxes, probs, landmarks = mtcnn.detect(image_array, landmarks=True)
    
    # Check if an eye was detected
    if boxes is not None:
        for landmark in landmarks:
            # Get the landmarks for the eyes
            left_eye = landmark[0]
            right_eye = landmark[1]
            
            # Crop and resize the image around the left eye
            roi_color = image_array[int(left_eye[1]-IMG_SIZE/3):int(left_eye[1]+IMG_SIZE/5), int(left_eye[0]-IMG_SIZE/4):int(left_eye[0]+IMG_SIZE/5)]
            if roi_color.size != 0:  # Check if the ROI is not empty
                try:
                    resized_array = cv2.resize(roi_color, (IMG_SIZE, IMG_SIZE))
                except:
                    return None
                
                return resized_array
            
    return None
# ------------------------------------------------------------


# Loading the models !!!
print('Seatbel Model is being loaded!')
print('Drowsiness Model is being loaded!')
print('Distraction Model is being loaded!')

seatbelt_model = load_model_fn('./seatbelt_Model_2.h5')
distraction_CNN_model = load_model_fn('./Distracted_Driver_Data_Augmentation.h5')
drowsiness_model = load_model_fn('./drowiness_new6.h5')

# ###################################################################################################

# 1- Seatbelt API
@app.route('/seatbelt', methods=['POST'])
def predict():
    try:
        message = request.get_json(force=True)
        encoded = message['image']
        decoded = base64.b64decode(encoded)
        image = Image.open(io.BytesIO(decoded))
    except:
        result = 'Unable to decode Image'
        return jsonify(result)
        
    try:
        processed_image = preprocess_image(image, target_size=(200, 200))
        prediction = seatbelt_model.predict(processed_image).tolist()
        print(prediction)
    except:
        result = 'The model could not predict the image'
        return jsonify(result)
    
    
     
    largest_value = max(prediction[0])
    print(largest_value)
    if largest_value > .60 :       
        response = {
            'prediction': {
                'seatbelt': prediction[0][0],
                'no seatbelt': prediction[0][1]
            }
        }
        result = max(response['prediction'], key=response['prediction'].get)
    elif largest_value < .60:
        result = 'Unknown'
    print(result)
    # Extract the label with the highest prediction value
    return jsonify(result)

# ###################################################################################################

# 2- #Drowsiness API
@app.route('/drowsiness', methods=['POST'])
def predict_drowsiness():
    message = request.get_json(force=True)
    encoded = message['image']
    decoded = base64.b64decode(encoded)
    image = Image.open(io.BytesIO(decoded))
    image_array = np.array(image)
    
    # Apply the preprocessing function
    face_image = crop_face(image_array=image_array)
    
    # Check if a face was detected

        
    if face_image is not None:
        # Reshape and normalize the image
        face_data = face_image.reshape(1, 145, 145, 3) / 255.0
    
        # Make prediction
        prediction_face = drowsiness_model.predict(face_data).tolist()
        largest_value = max(prediction_face[0])
        mapping_list = ['yawn', 'no_yawn', 'Closed', 'Open']
        
        if largest_value > .60:
            index_of_largest_value = np.argmax(prediction_face)
            
            if index_of_largest_value >1:
                result1 = 'Unknown'
            elif index_of_largest_value <= 1:
                result1 = mapping_list[index_of_largest_value]
        elif largest_value < .60:
            result1 = 'Unknown'
    else:
        result1 = "No faces detected in the image."
    
    # ------------------------------------------------------------
    
    # Apply the preprocessing function
    eye_image = crop_eye(image_array=image_array)
    
    # Check if a face was detected
    if eye_image is not None:
        
        # Reshape and normalize the image
        eye_data = eye_image.reshape(1, 145, 145, 3) / 255.0
    
        # Make prediction
        prediction_eye = drowsiness_model.predict(eye_data).tolist()
        largest_value = max(prediction_eye[0])
        mapping_list = ['yawn', 'no_yawn', 'Closed', 'Open']
        
        
    
        # Map prediction to output
        mapping_list = ['yawn', 'no_yawn', 'Closed', 'Open']
        if largest_value > .60:
            index_of_largest_value = np.argmax(prediction_face)
            
            if index_of_largest_value < 2:
                result2 = 'Unknown'
            elif index_of_largest_value >=2:
                result2 = mapping_list[index_of_largest_value]
        elif largest_value < .60:
            result2 = 'Unknown'
    
    else:
        result2 = "No Eyes detected in the image."

       
    response = {
        "Face_Prediction": result1,
        "Eye_Prediction": result2
    }
    

    return jsonify(response)

# ###################################################################################################

# 3- Distraction API
@app.route('/distraction', methods=['POST'])
def predict_driver_distraction():
    try:
        message = request.get_json(force=True)
        encoded = message['image']
        decoded = base64.b64decode(encoded)
        image = Image.open(io.BytesIO(decoded))
    except:
        result = 'Unable to decode message'
        return jsonify(result)
    
    processed_image = preprocess_image(image, target_size=(224, 224))
    
    try:
        prediction = distraction_CNN_model.predict(processed_image).tolist()
        print(prediction)
    except:
        result = 'The model could not predict the Image'
        return jsonify(result)
    
    label = ['normal driving',
             'texting - right',
             'talking on the phone - right',
             'texting - left',
             'talking on the phone - left',
             'operating the radio',
             'drinking',
             'reaching behind',
             'hair and makeup',
             'talking to passenger']
    
    largest_value = max(prediction[0])
    if largest_value > .60 :
        max_index = np.argmax(prediction)
        result = label[max_index]
    elif largest_value < .60:
        result = 'Unknown'


    return jsonify(result)

# ###################################################################################################

if __name__ == "__main__":
    app.run()

Seatbel Model is being loaded!
Drowsiness Model is being loaded!
Distraction Model is being loaded!
Model loaded!
Model loaded!
Model loaded!
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


1/1 [==============================] - 0s 50ms/step


127.0.0.1 - - [18/Sep/2023 19:48:26] "POST /drowsiness HTTP/1.1" 200 -


1/1 [==============================] - 0s 52ms/step


...

127.0.0.1 - - [18/Sep/2023 20:05:49] "POST /drowsiness HTTP/1.1" 200 -


1/1 [==============================] - 0s 53ms/step


127.0.0.1 - - [18/Sep/2023 20:06:17] "POST /drowsiness HTTP/1.1" 200 -


1/1 [==============================] - 0s 46ms/step


127.0.0.1 - - [18/Sep/2023 20:06:54] "POST /drowsiness HTTP/1.1" 200 -
